In [1]:
import pandas as pd

In [7]:
df=pd.read_csv('manifesto (2).csv',encoding='utf-8')

In [30]:
df=df.to_string()

In [36]:
df=df.replace('\\n',' ')
df=df.replace('Unnamed: 0 ','')
df

"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [9]:
df1=pd.read_csv('pre_final.csv')

In [10]:
df1

,Title,Date_published,Link,OP_ED,Similarity_BJP_2019
0,The road ahead: Government must fix the financ...,"December 2, 2019, 6:00 AM IST",https://timesofindia.indiatimes.com/blogs/toi-...,India’s gross domestic product in the July-Sep...,0.113011
1,Hyderabad horror: Widespread lack of safe publ...,"December 2, 2019, 5:00 AM IST",https://timesofindia.indiatimes.com/blogs/toi-...,The 27-year-old government veterinary doctor w...,0.096943
2,Populist potpourri: Maharashtra coalition must...,"November 30, 2019, 2:00 AM IST",https://timesofindia.indiatimes.com/blogs/toi-...,"Maharashtra’s new coalition government, dubbed...",0.105926
3,Wake up with makeup: How a teen vlogger school...,"November 30, 2019, 2:00 AM IST",https://timesofindia.indiatimes.com/blogs/toi-...,Makeup videos are a hardy perennial of the onl...,0.066813
4,Karvy Stock Broking problems show that the cap...,"November 29, 2019, 1:21 PM IST",https://timesofindia.indiatimes.com/blogs/toi-...,The narrative about the next generation of str...,0.102166
...,...,...,...,...,...
495,Tribal doctor’s suicide could have a caste factor,"May 29, 2019, 12:55 PM IST",https://timesofindia.indiatimes.com/blogs/toi-...,"The failure of mahagathbandhan, the alliance o...",0.099753
496,Didi in trouble: Mamata must cool political te...,"May 29, 2019, 12:51 PM IST",https://timesofindia.indiatimes.com/blogs/toi-...,The stunning growth of BJP in Bengal in the Lo...,0.080350
497,Front foot play: All BJP leaders should bat li...,"May 29, 2019, 2:07 AM IST",https://timesofindia.indiatimes.com/blogs/toi-...,Newly minted BJP MP Gautam Gambhir says he wil...,0.087867
498,Bimstec focus: Modi’s swearing-in will reflect...,"May 29, 2019, 2:04 AM IST",https://timesofindia.indiatimes.com/blogs/toi-...,"In a change of scene, both strategic and polit...",0.116472


In [13]:
from gensim.models.keyedvectors import KeyedVectors
model_path = 'GoogleNews-vectors-negative300.bin'
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [24]:
import numpy as np


class DocSim:
    def __init__(self, w2v_model, stopwords=None):
        self.w2v_model = w2v_model
        self.stopwords = stopwords if stopwords is not None else []

    def vectorize(self, doc: str) -> np.ndarray:
        """
        Identify the vector values for each word in the given document
        :param doc:
        :return:
        """
        doc = doc.lower()
        words = [w for w in doc.split(" ") if w not in self.stopwords]
        word_vecs = []
        for word in words:
            try:
                vec = self.w2v_model[word]
                word_vecs.append(vec)
            except KeyError:
                # Ignore, if the word doesn't exist in the vocabulary
                pass

        # Assuming that document vector is the mean of all the word vectors
        # PS: There are other & better ways to do it.
        vector = np.mean(word_vecs, axis=0)
        return vector

    def _cosine_sim(self, vecA, vecB):
        """Find the cosine similarity distance between two vectors."""
        csim = np.dot(vecA, vecB) / (np.linalg.norm(vecA) * np.linalg.norm(vecB))
        if np.isnan(np.sum(csim)):
            return 0
        return csim

    def calculate_similarity(self, source_doc, target_docs=None, threshold=0):
        """Calculates & returns similarity scores between given source document & all
        the target documents."""
        if not target_docs:
            return []

        if isinstance(target_docs, str):
            target_docs = [target_docs]

        source_vec = self.vectorize(source_doc)
        results = []
        for doc in target_docs:
            target_vec = self.vectorize(doc)
            sim_score = self._cosine_sim(source_vec, target_vec)
            if sim_score > threshold:
                results.append({"score": sim_score, "doc": doc})
            # Sort results by score in desc order
            results.sort(key=lambda k: k["score"], reverse=True)

        return results

In [26]:

ds = DocSim(w2v_model)

In [37]:
source_doc = df
for i in range(len(df1['OP_ED'])):
        target_docs = df1['OP_ED'][i]
        sim_scores = ds.calculate_similarity(source_doc, target_docs)

        print(sim_scores)

[{'score': 0.88979334, 'doc': 'India’s gross domestic product in the July-September quarter grew 4.5%, the slowest pace of quarterly growth since the beginning of 2013. This was the sixth consecutive quarter where the sequential growth rate was lower than the preceding one. The trend highlights the extent and duration of the loss in economic momentum. A sharp recovery from the current level is unlikely. Another data set, which captures the performance of eight core infrastructure industries, contracted by 5.8% in October. To underline, data suggests that the economic slowdown is broad and deep.Growth rate for the July-September quarter did not come as a shock. Multiple indicators pointed to a continuation of the slowdown. Of particular concern is the pattern of growth. Manufacturing contracted by 1%. To an extent, it has been adversely affected by lacklustre global trade. But there are many constraints which are entirely domestic in nature. Performance of manufacturing is worrisome fro

[{'score': 0.89067656, 'doc': 'The government opened up a controversial route for political funding in 2017, with the introduction of electoral bonds. Their genesis is back in the news, following a two-year effort of an RTI activist to ferret out documents relating to these instruments. Electoral bonds allow a potential donor to fund a chosen political party under the cloak of anonymity, to facilitate which changes were made to relevant sections of the Companies Act. The bonds are to be issued only through State Bank of India, which is aware of the identity of the donor. Given the weak institutional checks and balances in India, this does decidedly tilt the field in favour of the ruling party of the day.Note that anonymity of the donor applies only from the perspective of the voter, not necessarily from that of the government. This renders the government’s justification of electoral bonds – that the secrecy is necessary to encourage companies to donate without the fear of political ret

[{'score': 0.8596546, 'doc': 'With the Rajya Sabha passing the Transgender Persons (Protection of Rights) Bill 2019, India is set to get its first law protecting the rights of the transgender community. The bill was passed by the Lok Sabha earlier in August, and its passage into law is certainly a big step forward. The legislation is ambiguous in that it ostensibly gives every transgender person the right to self-perceived identity, yet it also stipulates that a transgender individual must apply to the district magistrate for a certificate of identity indicating transgender status. In case the person undergoes surgery to change gender to male or female, then a revised certificate will have to be obtained later.There are things to be said for and against the latter. It could be argued that it opens up transgender persons to intrusive scrutiny and robs them of agency. On the other hand, allowing transgender people to simply self-identify, as many on the left argue, can open up a can of w

[{'score': 0.87368584, 'doc': 'With NCP chief Sharad Pawar claiming that most of his MLAs barring a few have returned to his fold, it is incumbent upon the three-day-old government in Maharashtra to prove its majority at the earliest. Given the current murky situation where rumours are constantly swirling about the fluctuating loyalties of opposition MLAs, only a floor test in the legislative assembly can certify the numbers on the side of BJP and Ajit Pawar’s NCP faction. However, chief minister Devendra Fadnavis has shown no urgency to face the house.This is where Supreme Court needs to step in and quickly order a floor test, to be held within a day or so. Further delay only opens avenues for horse trading. Hung assemblies by their very nature are an invitation for unorthodox political arrangements but a red line is crossed when money power and intimidatory tactics are used. Unfortunately, constitutional authorities like the Governor and Speaker have disappointed in upholding convent

[{'score': 0.85981387, 'doc': 'On the outskirts of west Delhi, first-generation Dalit entrepreneur Subhash Singh Grover has earned a place for himself in the crane industry. A year back, he received an offer from a company to supply two cranes, both 250 metric tons in strength. He needed Rs 5 crore to commence production. Without any assets to pledge, he took a Rs 5 crore unsecured loan from a consortium of lenders at the interest rate of 15%. Since its birth in 2005, his company has spent over Rs 10 crore on interest.Another first-generation Dalit entrepreneur from Delhi, NK Chandan makes smart electric meter covers, junction boxes and bus bars for power companies. Since 2013, he has spent over Rs 5 crore paying interest. He once took a large sum from a private lender at an interest rate of 36%.The list of Dalit businessmen spending mind-boggling sums of money paying interest to usurious private lenders is long. Ahmedabad based Ratibhai Makwana trades in plastics. The Makwana group sp

...

[{'score': 0.9087408, 'doc': 'Quality public transport is a great social, economic and environmental good. But for far too long poor options on this front have robbed Indians of opportunities, as if we are trapped in a creaky time warp. For example, our railways still resembles its British Raj avatar more than 21st century peers in China or Japan. One indicator of change finally picking up pace is that 10 cities now have functional metro rail networks and more are lined up to hop on soon. Any retardation of this progress will be nothing less than tragic.Unfortunately, Delhi chief minister Arvind Kejriwal has plumped for the old povertarian approach to building infrastructure by proposing that women should travel for free in city public transport, including the metro. Men will of course cry foul, and in time even they would get to travel free. Kejriwal has offered a subsidy for this, a fixed amount that is supposed to compensate for the open-ended commitment that public transport includ

[{'score': 0.8742696, 'doc': 'The monsoon is running about a week late, and is expected to hit Kerala on Thursday. Its progress thereafter will be tracked zealously. After all 42% of the country is abnormally dry at present, water levels in most western and southern states have dropped to less than the average of the last decade, and a severe heat wave is sweeping through the north and northwest.As many as 10 places in India currently figure among the 15 hottest in the world. Temperature has hit 45 degrees Celsius in several cities like Delhi and Lucknow. In Rajasthan it has neared a scorching 49 degrees. There is little respite even in hill stations like Shimla and Mussoorie, which are also seeing above normal temperatures. In many places this intense heat spell is expected to last the week.So this is a week in which state governments must pay special attention to the NDMA guidelines for managing heat waves. This includes reordering work and school timings as needed, setting up public

[{'score': 0.8022902, 'doc': 'In its latest act of unaccountable and unrestrained authoritarianism, Pakistan army for the first time has handed death sentence to one of its high-ranking retired officers and a civil doctor along with 14 years rigorous imprisonment to a retired general on charges of espionage. The army refused to divulge the details of the offences and the process of trial of the officers. As per news reports from Pakistan, in the last two years, the army has punished around 400 officers of various ranks for undisclosed offences.Though it was known from the beginning that the military backed Imran Khan would be a weak prime minister, it is becoming increasingly clear that army chief General Qamar Javed Bajwa is the most powerful leader in the country. The army also continues to be the most opaque institution in Pakistan.Until now, Pakistan had earned the dubious distinction of arresting, torturing, trying without due process and wrongfully convicting Indians on the charg

[{'score': 0.82453215, 'doc': 'On Sunday, Pakistani security forces killed at least eight protestors of Pashtun Tahafuz Movement (PTM) in Waziristan and took into custody one of its prominent leaders, who is also a member of Pakistan’s parliament. Accusing PTM of being a proxy of Afghanistan and India, Pakistan army, known for its dictatorial history, has threatened to mercilessly repress the Pashtun rights protest movement further.PTM, a peaceful campaign run by Pashtun people especially youth against violations of their human rights in Khyber Pakhtunkhwa and Balochistan, was founded in 2014. Led by 24-year-old Manzoor Ahmad Pashteen, the movement gained momentum in January 2018 over an extrajudicial killing of one of the aspiring models of the Pashtun people. Since then, a few more Pashtun people have been killed by extrajudicial means.Unlike the Balochistan movement which is violent and separatist, Pashtun rights movement has been peaceful but verbally very critical of the Pakistani

[{'score': 0.8693738, 'doc': 'Newly minted BJP MP Gautam Gambhir says he will “continue to live in black and white”. Some things are indeed black and white. Netas across the aisles should stop turning black into white because they think it will help their party’s cause. Political whataboutery with hate crimes impedes police investigations and contributes to abysmal law and order conditions in the country.This particular case involves Mohammad Barkat Aalam allegedly being assaulted on Saturday night in Gurgaon, and also forced to chant ‘Jai Sri Ram’. Gambhir’s response to the incident was commonsensical: “It is deplorable. Exemplary action needed by Gurugram authorities.” That he got trolled for this may not be such a surprise, given that mindlessness is the quality that most distinguishes trolls. More questionable was the sledging by his partymen, who see a conspiracy by anti-BJP forces to paint crimes in communal colours. In this case, the conspiracy seems to extend to a freshly elect